In [31]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

,PATIENT ID,Label
0,TCGA-5M-AAT6,1
1,TCGA-A6-2686,0
2,TCGA-A6-5661,0
3,TCGA-A6-5665,0
4,TCGA-A6-6653,1
...,...,...
529,TCGA-CM-4748,1
530,TCGA-D5-6923,0
531,TCGA-DC-6156,0
532,TCGA-DY-A1DE,0


In [36]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub
# Define image preprocessing parameters
datagen = ImageDataGenerator(
    rescale=1.0/255,  # Normalize pixel values
    rotation_range=20,  # Data augmentation options
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Split the data into training and validation
)

# Load and preprocess the training dataset
train_generator = datagen.flow_from_directory(
    '/kaggle/input/task02/train',
    target_size=(224, 224),  # Adjust to match your model's input size
    batch_size=32,
    class_mode='categorical',  # Change to 'binary' for binary classification
    subset='training'
)

# Load and preprocess the validation dataset
validation_generator = datagen.flow_from_directory(
    '/kaggle/input/task02/vad',
    target_size=(224, 224),  # Adjust to match your model's input size
    batch_size=32,
    class_mode='categorical',  # Change to 'binary' for binary classification
    subset='validation'
)


Found 153 images belonging to 2 classes.
Found 37 images belonging to 2 classes.


In [37]:
# Load the pre-trained ViT model
model = tf.keras.Sequential([
    hub.KerasLayer("https://www.kaggle.com/models/spsayakpaul/vision-transformer/frameworks/TensorFlow2/variations/vit-b16-classification/versions/1")
])

# Modify the model for your classification task (e.g., change the output layer)
# You should replace this with the number of classes in your dataset
num_classes = len(train_generator.class_indices)
print(num_classes)
model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',  # Use 'binary_crossentropy' for binary classification
    metrics=['accuracy']
)

# Train the model
model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=1  # You can adjust the number of epochs
)


2
5/5 [==============================] - 26s 4s/step - loss: 2.3299 - accuracy: 0.3595 - val_loss: 2.3299 - val_accuracy: 0.3243


In [41]:
import tensorflow as tf
import cv2  # Assuming you're using OpenCV for image preprocessing

# Load the pre-trained model (already loaded in your code)

# Load and preprocess the input image
image_path = 'TCGA-AA-3952-01Z-00-DX1.6a51a689-74cb-4204-9f7a-f5e3fc55fb2d_(11705,24607).jpg'
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Ensure image is in RGB format
image = cv2.resize(image, (224, 224))  # Resize to match model input size
image = image / 255.0  # Normalize pixel values (if the model expects it)

# Make a prediction
predictions = model.predict(tf.expand_dims(image, axis=0))

# Get the class with the highest probability
predicted_class = int(tf.argmax(predictions, axis=1)[0])

1/1 [==============================] - 0s 24ms/step


In [42]:
print(predicted_class)

0


In [40]:
# Get the class indices as integers
class_indices = list(train_generator.class_indices.values())

# Print the class indices
print(class_indices)


[0, 1]


In [29]:
generator.classes

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
      dtype=int32)